In [1]:
import json
import os
import shutil
import tempfile
from deepdiff import DeepDiff
import zipfile
from pathlib import Path
from log import read_eval_log, write_eval_log
import time

log_json = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "log_formats.json"))
log_eval = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "log_formats.eval"), validate=True)
log_eval_skip = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "log_formats.eval"), validate=False)

In [2]:
def nullify_ids(obj):
    """Recursively set all ID fields to a consistent value for comparison"""
    if isinstance(obj, dict):
        obj_copy = {}
        for k, v in obj.items():
            if k in ('id', 'id_', 'eval_id', 'run_id', 'task_id', 'uuid', 'span_id'):
                obj_copy[k] = "NORMALIZED_ID"  # Use consistent placeholder
            else:
                obj_copy[k] = nullify_ids(v)
        return obj_copy
    elif isinstance(obj, list):
        return [nullify_ids(item) for item in obj]
    elif hasattr(obj, '__dict__'):  # Handle Pydantic objects
        # Convert to dict first, then process
        if hasattr(obj, 'model_dump'):
            return nullify_ids(obj.model_dump())
        else:
            return nullify_ids(obj.__dict__)
    else:
        return obj

def compare_without_ids(obj1, obj2):
    """Compare objects with all IDs normalized"""
    obj1_clean = nullify_ids(obj1)
    obj2_clean = nullify_ids(obj2)
    
    diff = DeepDiff(obj1_clean, obj2_clean, ignore_order=True)
    return diff

# Usage
diff = compare_without_ids(log_eval, log_eval_skip)
if diff:
    print("Differences found (excluding IDs):")
    print(diff)
else:
    print("✅ Objects are identical (excluding IDs)")

Differences found (excluding IDs):
{'dictionary_item_removed': ["root['samples'][0]['events'][5]['output']['choices'][0]['message']['internal']", "root['samples'][0]['events'][5]['output']['choices'][0]['message']['tool_calls']", "root['samples'][0]['events'][5]['output']['choices'][0]['message']['model']"]}


/home/ubuntu/inspect_ai/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `SampleInitEvent` - serialized value may not be as expected [input_value=ModelEvent(id_=None, span...None, working_time=None), input_type=ModelEvent])
  PydanticSerializationUnexpectedValue(Expected `SampleLimitEvent` - serialized value may not be as expected [input_value=ModelEvent(id_=None, span...None, working_time=None), input_type=ModelEvent])
  PydanticSerializationUnexpectedValue(Expected `SandboxEvent` - serialized value may not be as expected [input_value=ModelEvent(id_=None, span...None, working_time=None), input_type=ModelEvent])
  PydanticSerializationUnexpectedValue(Expected `StateEvent` - serialized value may not be as expected [input_value=ModelEvent(id_=None, span...None, working_time=None), input_type=ModelEvent])
  PydanticSerializationUnexpectedValue(Expected `StoreEvent` - serialized value may not

## New Profiling

In [3]:
#!/usr/bin/env python3

import sys
import time
import cProfile
import pstats
import io
sys.path.insert(0, 'src')

from inspect_ai.log import read_eval_log

def profile_large_log():
    """Profile reading a large log file to find bottlenecks."""
    
    print("Profiling large log file reading...")
    
    # Profile validate=False path
    pr = cProfile.Profile()
    pr.enable()
    
    start = time.time()
    log_eval = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=False)
    duration = time.time() - start
    
    pr.disable()
    
    print(f"Read time: {duration:.2f}s")
    print(f"Samples: {len(log_eval.samples) if log_eval.samples else 0}")
    
    # Analyze profile
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats(20)  # Top 20 functions
    
    print("\nTop bottlenecks:")
    print("=" * 80)
    print(s.getvalue())

if __name__ == "__main__":
    import os
    profile_large_log()

Profiling large log file reading...
Read time: 249.62s
Samples: 400

Top bottlenecks:
         191606376 function calls (187901769 primitive calls) in 249.616 seconds

   Ordered by: cumulative time
   List reduced from 482 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       48    0.002    0.000  310.210    6.463 /home/ubuntu/inspect_ai/.venv/lib/python3.12/site-packages/nest_asyncio.py:100(_run_once)
      2/1    0.000    0.000  249.615  249.615 /home/ubuntu/inspect_ai/src/inspect_ai/log/_file.py:211(read_eval_log)
      2/1    0.000    0.000  249.615  249.615 /home/ubuntu/inspect_ai/src/inspect_ai/_util/_async.py:108(run_coroutine)
      2/1    0.000    0.000  249.615  249.615 /home/ubuntu/inspect_ai/.venv/lib/python3.12/site-packages/nest_asyncio.py:25(run)
      2/1    0.000    0.000  249.615  249.615 /home/ubuntu/inspect_ai/.venv/lib/python3.12/site-packages/nest_asyncio.py:86(run_until_complete)
     53/4    0.000    0.000

In [3]:
start = time.time()
non_validated_big_bad_log_eval = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=False)
print(f"validate=True: {time.time() - start:.2f}s")

validate=True: 155.62s


In [3]:
start = time.time()
non_validated_big_bad_log_eval = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=False)
print(f"validate=True: {time.time() - start:.2f}s")

validate=True: 94.93s


## Original Profiling

In [4]:
start = time.time()
big_bad_log_eval = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=True)
print(f"validate=True: {time.time() - start:.2f}s")

validate=True: 259.99s


In [2]:
start = time.time()
big_bad_log_eval_skip = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=False)
print(f"validate=False: {time.time() - start:.2f}s")

validate=False: 243.02s


In [2]:
start = time.time()
big_bad_log_eval_skip = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=False)
print(f"validate=False: {time.time() - start:.2f}s")

validate=False: 65.02s


In [3]:
start = time.time()
big_bad_log_eval_skip = read_eval_log(os.path.join("..", "..", "tests", "log", "test_eval_log", "mistral_cybench_1_temp.eval"), validate=False)
print(f"validate=False: {time.time() - start:.2f}s")

validate=False: 65.11s


In [9]:
assert log_json.model_dump_json() == log_eval.model_dump_json(), (
    "Logs in different formats are not identical"
)

AssertionError: Logs in different formats are not identical

In [7]:
assert log_eval_skip.model_dump_json() == log_eval.model_dump_json(), (
    "Logs in different formats are not identical"
)

In [8]:
log_json.model_dump_json()

'{"version":2,"status":"success","eval":{"eval_id":"en23VHYVcj2iF3Bng6W4Bn","run_id":"BpiPFmbVcMCRUAXv8ovCTx","created":"2024-10-15T12:53:13-04:00","task":"popularity","task_id":"9eTnG3tV8jGRYoTPMH54Ee","task_version":0,"task_file":"examples/popularity.py","task_registry_name":null,"task_attribs":{},"task_args":{},"solver":null,"solver_args":null,"tags":null,"dataset":{"name":"popularity","location":"example://popularity","samples":100,"sample_ids":[1],"shuffled":false},"sandbox":null,"model":"ollama/llama3.1","model_generate_config":{"max_retries":null,"timeout":null,"max_connections":null,"system_message":null,"max_tokens":null,"top_p":null,"temperature":null,"stop_seqs":null,"best_of":null,"frequency_penalty":null,"presence_penalty":null,"logit_bias":null,"seed":null,"top_k":null,"num_choices":null,"logprobs":null,"top_logprobs":null,"parallel_tool_calls":null,"internal_tools":null,"max_tool_output":null,"cache_prompt":null,"reasoning_effort":null,"reasoning_tokens":null,"reasoning_

In [9]:
log_eval_skip.model_dump_json()

'{"version":2,"status":"success","eval":{"eval_id":"en23VHYVcj2iF3Bng6W4Bn","run_id":"BpiPFmbVcMCRUAXv8ovCTx","created":"2024-10-15T12:53:13-04:00","task":"popularity","task_id":"9eTnG3tV8jGRYoTPMH54Ee","task_version":0,"task_file":"examples/popularity.py","task_registry_name":null,"task_attribs":{},"task_args":{},"solver":null,"solver_args":null,"tags":null,"dataset":{"name":"popularity","location":"example://popularity","samples":100,"sample_ids":[1],"shuffled":false},"sandbox":null,"model":"ollama/llama3.1","model_generate_config":{"max_retries":null,"timeout":null,"max_connections":null,"system_message":null,"max_tokens":null,"top_p":null,"temperature":null,"stop_seqs":null,"best_of":null,"frequency_penalty":null,"presence_penalty":null,"logit_bias":null,"seed":null,"top_k":null,"num_choices":null,"logprobs":null,"top_logprobs":null,"parallel_tool_calls":null,"internal_tools":null,"max_tool_output":null,"cache_prompt":null,"reasoning_effort":null,"reasoning_tokens":null,"reasoning_

In [15]:
import json
import difflib
from pprint import pprint

try:
    assert log_json.model_dump() == log_eval.model_dump(), (
        "Logs in different formats are not identical"
    )
except AssertionError:
    print("Differences found between log_json and log_eval:")
    
    # Convert to formatted JSON strings for better diff visualization
    json_str = json.dumps(log_json.model_dump(), indent=2, sort_keys=True)
    eval_str = json.dumps(log_eval.model_dump(), indent=2, sort_keys=True)
    
    # Show diff in unified format
    diff = difflib.unified_diff(
        json_str.splitlines(),
        eval_str.splitlines(),
        fromfile='log_json',
        tofile='log_eval',
        lineterm=''
    )
    
    for line in diff:
        print(line)
    

    
    raise  # Re-raise the assertion error
    

Differences found between log_json and log_eval:
--- log_json
+++ log_eval
@@ -229,7 +229,7 @@
             "tools": []
           },
           "timestamp": "2024-10-15T16:53:13.264282+00:00",
-          "working_start": 261562.454824458
+          "working_start": 2151527.298716833
         },
         {
           "action": "begin",
@@ -239,7 +239,7 @@
           "span_id": null,
           "timestamp": "2024-10-15T16:53:13.264623+00:00",
           "type": "solver",
-          "working_start": 261562.455447545
+          "working_start": 2151527.29885825
         },
         {
           "changes": [
@@ -280,7 +280,7 @@
           "pending": null,
           "span_id": null,
           "timestamp": "2024-10-15T16:53:13.265538+00:00",
-          "working_start": 261562.455573666
+          "working_start": 2151527.298872
         },
         {
           "action": "end",
@@ -290,7 +290,7 @@
           "span_id": null,
           "timestamp": "2024-10-15T16:53:13.265563+00:00",
     

AssertionError: Logs in different formats are not identical